# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 24 нояб

__Жесткий дедлайн: 27 нояб


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов, 2 балл за доп вопросы, если возникнут, если вопросов не возникло, считаем, что 2 балла вы получили 

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

517401

In [2]:
emails.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
test_emails = emails.tail()
test_emails = test_emails.copy()

for test_email in test_emails['message'].values.tolist():
    print(test_email)
    print('\n')

Message-ID: <26807948.1075842029936.JavaMail.evans@thyme>
Date: Wed, 28 Nov 2001 13:30:11 -0800 (PST)
From: john.zufferli@enron.com
To: kori.loibl@enron.com
Subject: Trade with John Lavorato
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JZUFFER>
X-To: Loibl, Kori </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Kloibl>
X-cc: 
X-bcc: 
X-Folder: \ExMerge - Zufferli, John\Sent Items
X-Origin: ZUFFERLI-J
X-FileName: john zufferli 6-26-02.PST

This is a trade with OIL-SPEC-HEDGE-NG (John Lavorato's book) and John Zufferli's book CAND-PWR-PR

CAND-PWR-PR buys from OIL-SPEC-HEDGE-NG Nymex Gas

Cal 03			38,500 MMBtu/day		$3.2978
Cal 04-Cal 05		35,000 MMBtu/day		$3.4482
Cal 06-Cal 10		40,000 MMBtu/day		$3.7369
Cal 11-Cal 15		18,500 MMBtu/Day		$4.2057

Please Confirm Receipt


John Z


Message-ID: <25835861.1075842029959.JavaMail.evans@thyme>
Date: Wed, 28 Nov 2001 12:47:48 -0800 (PST)
From: john.zufferli@enro

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [4]:
from email import message_from_string
import re

def get_message_body(message: str) -> str:
    # Парсим письмо
    email_message = message_from_string(message)
    # Получаем тело письма
    return email_message.get_payload(decode=True).decode(email_message.get_content_charset() or 'utf-8')

def clear_message_body(message: str) -> str:
    # Получаем тело письма
    body = get_message_body(message)

    # Удаляем строки с ключами To:, From:, RE:, и подобными
    meta_info_pattern = r'(To:|From:|Subject:|CC:|BCC:|RE:|Sent:|cc:|RE:).*$'
    body = re.sub(meta_info_pattern, '', body, flags=re.MULTILINE)

    # Удаляем текст, идущий после -----Original Message----- до первой пустой строки
    original_message_pattern = r'-----Original Message-----.*?\n\n'
    body = re.sub(original_message_pattern, '', body, flags=re.DOTALL)

    # Удаляем адреса электронной почты
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'
    body = re.sub(email_pattern, '', body)

    # Удаляем даты (формат: день, месяц, год, например: 14 May 2001 или 28/11/2001)
    date_pattern = r'\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{1,2}\s+[A-Za-z]{3,9}\s+\d{2,4})\b'
    body = re.sub(date_pattern, '', body)

    # Удаляем время (формат: 14:30, 8:17 AM, 16:39:00)
    time_pattern = r'\b(?:[01]?\d|2[0-3]):([0-5]?\d)(?::([0-5]?\d))?\b|\d{1,2}:\d{2}\s?(AM|PM)?\b'
    body = re.sub(time_pattern, '', body)

    # Удаляем disclaimers (юридические уведомления, например текст после *****)
    disclaimer_pattern = r'\n\*+.*?This e-mail.*?(?:\n\*+|\Z)'
    body = re.sub(disclaimer_pattern, '', body, flags=re.DOTALL)

    # Убираем дополнительные пробелы
    body = re.sub(r'\s+', ' ', body).strip()

    return body

In [5]:
# Протестируем на 5 элементах датасета
test_emails['message'] = test_emails['message'].apply(clear_message_body)
# Дропнем ненужный столбец
test_emails = test_emails.drop('file', axis = 1)

test_emails

,message
517396,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,Some of my position is with the Alberta Term b...
517398,"2 Morning John, I'm still working on the mini-..."
517399,Analyst Rank Stephane Brodeur 1 Chad Clark 1 I...
517400,i think the YMCA has a class that is for peopl...


In [6]:
for msg in test_emails['message'].values:
    print(msg)

This is a trade with OIL-SPEC-HEDGE-NG (John Lavorato's book) and John Zufferli's book CAND-PWR-PR CAND-PWR-PR buys from OIL-SPEC-HEDGE-NG Nymex Gas Cal 03 38,500 MMBtu/day $3.2978 Cal 04-Cal 05 35,000 MMBtu/day $3.4482 Cal 06-Cal 10 40,000 MMBtu/day $3.7369 Cal 11-Cal 15 18,500 MMBtu/Day $4.2057 Please Confirm Receipt John Z
Some of my position is with the Alberta Term book, I will send you only the positions that I have directly with ENA.
2 Morning John, I'm still working on the mini-PRC for Lavo. Sean Lalani has not yet been ranked and rumour has it that he reports to you now. Can you confirm and send me a number. Thanks!
Analyst Rank Stephane Brodeur 1 Chad Clark 1 Ian Cooke 3 Lon Draper 1 Fabian Taylor 2 Carlos Torres 3 Ryan Watt 1 Associate Cooper Richey 1
i think the YMCA has a class that is for people recovering from heart-attacks i remeber something about that i don't know about the heart classes. i'll look into it, but her dr (ravi) isn't offering up any suggestions or anythi

In [7]:
emails['message'] = emails['message'].apply(clear_message_body)
# Дропнем ненужный столбец
emails = emails.drop('file', axis=1)
# сохраним полученный датасет
emails.to_csv('cleared_emails.csv')


In [9]:
emails.sample()['message'].values

array(['Edison International CFO Disputes \'Technical Insolvency\' Claim By Mark Golden Dow Jones Energy Service (Copyright (c) 2000, Dow Jones & Company, Inc.) NEW YORK -(Dow Jones)- Edison International\'s treasurer and chief financial officer, Ted Craver, disputed Tuesday a claim made last week that California\'s utilities risk "technical insolvency" in 2001. Since May, California\'s three investor-owned utilities have been spending far more to purchase needed electricity than they are getting paid by customers. As a result, the undercollections in the "transition revenue account" of Edison\'s regulated utility unit, Southern California Edison, grew by $1.97 billion between May 1 and August 31, the company reported to the Securities Exchange Commission. An article Wednesday in the Wall Street Journal, which is published by Dow Jones & Co., said that "if the cost of wholesale power continues to exceed the price these utilities are allowed to bill their customers, as currently seems l

In [2]:
import pandas as pd

emails = pd.read_csv('cleared_emails.csv')

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [5]:
%load_ext autoreload
%autoreload 2

from typing import List

class PrefixTreeNode:
    def __init__(self):
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        
        for word in vocabulary:
            current_node = self.root
            for char in word:
                if char not in current_node.children:
                    current_node.children[char] = PrefixTreeNode()
                current_node = current_node.children[char]
            current_node.is_end_of_word = True

    def search_prefix(self, prefix: str) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """
        current_node = self.root
        
        for char in prefix:
            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                return []

        def get_bottom_prefixes(node: PrefixTreeNode, path: str) -> List[str]:
            results = []
            if node.is_end_of_word:
                results.append(path)
            for char, child in node.children.items():
                results.extend(get_bottom_prefixes(child, path + char))
            return results

        return get_bottom_prefixes(current_node, prefix)
        

In [6]:
%load_ext autoreload
%autoreload 2

vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [7]:
class WordCompletor:
    def __init__(self, corpus: List[str]):
        """
        corpus: list – корпус текстов
        """
        self.word_count_dict = {}
        
        self.words_count = 0
        
        for corpus_item in corpus:
            self.words_count += len(corpus_item)
            for word in corpus_item:
                if word in self.word_count_dict.keys():
                    self.word_count_dict[word] += 1
                else:
                    self.word_count_dict[word] = 1

        self.word_count_dict = {key: value / self.words_count for key, value in self.word_count_dict.items()}
        
        self.prefix_tree = PrefixTree(self.word_count_dict.keys())
        

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words = self.prefix_tree.search_prefix(prefix)
        
        probs = [self.word_count_dict[word] for word in words]
        
        return words, probs

In [8]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [9]:
from typing import List, Tuple
from collections import Counter

class NGramLanguageModel:
    def __init__(self, corpus, n):
        # максимальная длинна N-граммы
        self.n = n 
        # Счётчик частот N-грамм
        self.ngram_counts = Counter()
        # Счётчик частот контекстов (первых n-1 слов)
        self.context_counts = Counter()
        
        # Построение N-грамм из корпуса
        for sentence in corpus:
            sentence_length = len(sentence)
            
            for word_index in range(sentence_length):
                for ngram_length in range(1, sentence_length - word_index + 1):  
                    ngram = tuple(sentence[word_index : word_index + ngram_length])
                    self.ngram_counts[ngram] += 1
                    # Контексты существуют только для n-грамм длиной > 1
                    if len(ngram) > 1:  
                        context = ngram[:-1]
                        self.context_counts[context] += 1
                
    def get_next_words_and_probs(self, prefix: list) -> (List[str], List[float]):
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов
        """
        
        # Возможные следующие слова
        next_words = []  
        # Вероятности этих слов
        probs = []  
        # Преобразуем prefix в кортеж для сопоставления с n-граммами
        context = tuple(prefix)  
        # Найти все n-граммы, начинающиеся с данного контекста
        for ngram, count in self.ngram_counts.items():

            if ngram[:-1] == context:  # Проверка, соответствует ли n-грамма данному контексту
                next_word = ngram[-1]
                next_words.append(next_word)
                
                # Вычисляем вероятность слова как P(next_word | context)
                context_count = self.context_counts[context]
                probs.append(count / context_count)

        return next_words, probs

In [10]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [17]:
from typing import List, Union

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """
        
        suggestions = []
        
        suggestion = []
        
        if(isinstance(text, str)):
            words = text.strip().split()
        elif(isinstance(text, list)):
            words = text
        else:
            raise ValueError('Недопустимый тип text')
        
        if(len(words) == 0):
            return []
        # получим последнее слово
        last_word = words[-1]
        # получаем варианты дополнения последнего слова до целого
        completor_words, completor_probs = self.word_completor.get_words_and_probs(last_word)
        # если completor_words вернул дополненные слова, то берем слово с наибольшей вероятностью
        last_word = completor_words[completor_probs.index(max(completor_probs))] if len(completor_words) > 0 else last_word
        # заменяем последнее слово
        words[-1] = last_word
        
        suggestion.append(last_word)
        
        n = self.n_gram_model.n
        # получаем контекст (n - 1) слов
        context = words[-(n - 1):]  if n > 1 else []
        
        for _ in range(n_words):
            # получаем предсказания следующих n_word слов
            n_gram_words, n_gram_probs = self.n_gram_model.get_next_words_and_probs(context)
            
            if len(n_gram_words) == 0:
                break
            # берем слово с макс вероятностью
            next_word = n_gram_words[n_gram_probs.index(max(n_gram_probs))]
            
            suggestion.append(next_word)
            # добавляем в контекст
            context = context[1:] + [next_word] if n > 1 else [next_word]
            
        suggestions.append(suggestion)
            
        return suggestions

In [18]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [19]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!